In [2]:
# encoding: utf-8
# import tensorflow as tf
import datetime,time
import pandas as pd
import json
from pymongo import MongoClient
import akshare as ak
from pandas import DataFrame

pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行

def get_trade_rank(market = 'SHF', date = None):
    if date is None:
        date = get_target_date(-1, "%Y-%m-%d")
    if market == 'SHF':
        return ak.get_shfe_rank_table(date)
    if market == 'DCE':
        return ak.get_dce_rank_table(date)
    if market == 'CZC':
        return ak.get_czce_rank_table(date)
    if market == "CFE":
        return ak.get_cffex_rank_table(date)
    return None, '不支持的市场类型'

if __name__ == '__main__':

    markets = ['CZC', 'SHF','CFE','DCE']#, 'CZC', 'SHF','CFE','DCE'
    # 连接数据库
    client = MongoClient('localhost', 27017)
    db = client.futures3
    position = db.position
    
    df=pd.DataFrame()
    for market in markets:
        begin = datetime.date(2020,10,19)
        # end = datetime.date(2020,2,25)
        end = datetime.date.today()

#         begin = DataFrame(list(position.find({}).sort([('_id', -1)]).limit(1)))
        # begin = begin['date'][0]
        # begin = time.strptime(begin, "%Y%m%d")
        # year, month, day = begin[:3]
        # begin = datetime.date(year, month, day)
        # begin = begin + datetime.timedelta(days=1)

        for i in range((end - begin).days + 1):
            day = begin + datetime.timedelta(days=i)
            days=day.strftime('%Y%m%d')
            try:
                df = get_trade_rank(market, date=days)
                # print(days, market)
                for key, value in df.items():
                    value['date'] = days
                    print(value)
                    value['symbol']=value['symbol'].str.upper()
                    value= value[value['long_party_name'].notna()]
                    # print(value)
                    # position.insert_many(json.loads(value.T.to_json()).values())
                    # print(json.loads(value.T.to_json()).values())
                    #去除具体合约。因汇总持仓有问题
#                     if market != 'CZC':
#                         print('insert into',key)
#                         # position.update_one(value, {'$set': value}, True)
#                         position.insert_many(json.loads(value.T.to_json()).values())

#                     else:
#                         value=value[value['symbol']==value['variety']]
# #                         value= value[value['long_party_name'].notna()]
#                         print('insert into',key)
#                         df1 = pd.DataFrame(value)
#         
                    df = df.append(df1)
#                         df=df.append(value)
                        # position.update_one(value, {'$set': value}, True)
                        # position.insert_many(json.loads(value.T.to_json()).values())
#                         print(json.loads(value.T.to_json()).values())

            except:
                print(days,market,'数据异常')
                continue
df

   rank vol_party_name     vol  vol_chg long_party_name long_open_interest  \
0     1           海通期货  77,518  -14,514            鲁证期货              8,021   
1     2           光大期货  22,475   -3,594            中信期货              7,515   
2     3           中信期货  20,672   -5,395            华泰期货              6,375   
3     4           徽商期货  19,456   -4,185            国泰君安              6,326   
4     5           华闻期货  17,171   -3,880            光大期货              6,060   
5     6           华泰期货  17,114   -3,606            东证期货              5,499   
6     7           东吴期货  16,141   -7,227            徽商期货              4,583   
7     8           东证期货  14,896   -6,157            方正中期              3,971   
8     9           华安期货  14,397   -1,340            宏源期货              3,824   
9    10           新湖期货  11,895    2,127            海通期货              3,565   
10   11           国泰君安  11,140     -607            长安期货              3,500   
11   12           中辉期货  10,869      457            银河期货         

    long_open_interest  long_open_interest_chg long_party_name  rank  \
0                 8043                    -291            中信期货     1   
1                 7586                     -68            国泰君安     2   
2                 5434                    -218            瑞银期货     3   
3                 5150                     -59            海通期货     4   
4                 2976                      37            一德期货     5   
5                 2777                     -52            申银万国     6   
6                 2455                      65            华泰期货     7   
7                 2264                    -155            广发期货     8   
8                 2158                      49            永安期货     9   
9                 2058                     155            上海东证    10   
10                1815                       1            中金期货    11   
11                1656                     120            银河期货    12   
12                1541                      55            浙商期货  

{}

In [48]:
 #所有会员
# party_name = value[value['date'] == end]
long_party_name = df['long_party_name']
short_party_name = df['short_party_name']
party_name = long_party_name.append(short_party_name).drop_duplicates()
#多空变化量求和
long = value.groupby(['date', 'variety', 'long_party_name'])['long_openIntr'].sum()
# print(long)
short = value.groupby(['date', 'variety', 'short_party_name'])['short_openIntr'].sum()
# # 合并
frames = [long, short]
position = pd.concat(frames, axis=1, sort=True).fillna(0).reset_index()
position

,level_0,level_1,level_2,long_openIntr,short_openIntr
0,20201019,EB,一德期货,2457.0,2542.0
1,20201019,EB,东吴期货,0.0,3850.0
2,20201019,EB,东海期货,0.0,6118.0
3,20201019,EB,东航期货,2976.0,0.0
4,20201019,EB,东证期货,4193.0,7221.0
5,20201019,EB,中信建投,4544.0,0.0
6,20201019,EB,中信期货,4675.0,2067.0
7,20201019,EB,中财期货,3001.0,3495.0
8,20201019,EB,中辉期货,2770.0,0.0
9,20201019,EB,五矿经易,0.0,2470.0


In [113]:
 # 字段更名
position = position.rename(columns={'level_0': 'date', 'level_1': 'variety', 'level_2': 'BrokerID'})

position['net_chg'] = position.apply(lambda x: x['short_openIntr'] - x['long_openIntr'], axis=1)
# position
party_names=['永安期货','海通期货','中信期货']

df = pd.DataFrame()
for i in party_names:
    try:
#     print(i)
        mem = position[position['BrokerID'] == i]
    
        
#         new_df = new_df.rename_axis(columns=None)
#         new_df = new_df
        df1 = pd.DataFrame(mem)
#         
        df = df.append(df1)
        
        
    except:
        print('?')
        continue

two_level_index_series = df.set_index(['variety','BrokerID'])['net_chg']
new_df = two_level_index_series.unstack()
new_df['total'] = new_df.apply(lambda x: x.sum(), axis=1)
new_df = new_df.rename_axis(columns=None).reset_index()
# 
new_df

,variety,中信期货,永安期货,total
0,EB,-2608.0,693.0,-1915.0


In [114]:
new_df.to_excel('E:\pp.xlsx',index = True)